<a href="https://colab.research.google.com/github/brian-ho/intro-to-urban-data/blob/main/Measure/M05_Rhino_files.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **05 Measure** Working with Rhino Files
```
    Class       ARCH 6131 Urban Design Methods / Skills / Tools 1
                The Gensler Family AAP NYC Center
                Fall 2023
                Monday 3:30 PM to 6 PM

    Instructor  Brian Ho
                brian@brian-ho.io
```

This notebook shows how you can read and write Rhinoceros 3DM files _programmatically_. This does not rerplace your use of Rhino as a desktop app: you will always need to work more directly as a designer. But being able to open and edit Rhino files is a powerful way to connect your data to your choice of drawing and desiign.

In this notebook will cover:
- Reading a Rhino file
- Accessing layers and geometry
- Creating layers and geometry
- Savinig a Rhino file

Almost all the functionality in this notebook depends on a Python library from McNeel called  `rhino3dm`. This exposes functionality to read and write all geometry types, and is based on an open standard. You can learn more about it [here](https://github.com/mcneel/rhino3dm).

⚠ **`rhino3dm` requires a custom build to run on Colab**

_If you are running this notebook on Colab, you'll need to install from a custom build I've created for the typical Colab environment using the cell below.  You will need to do this each time you open the notebook on a new Colab runtime. If you are runninig this locally you can install via `pip` and it's included in the repository's `requirements.txt`._

In [5]:
!pip install https://intro-to-urban-data.s3.amazonaws.com/rhino3dm-8.0.0b2-cp310-cp310-linux_x86_64.whl --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 44.4 MB/s eta 0:00:00


In [6]:
# After installing the custom build, you can import it into the notebook.
import rhino3dm

Connect to drive to access your custom files.

In [7]:
from google.colab import drive

drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
EXAMPLE_DATA_DIRECTORY = (
    "/content/drive/MyDrive/Cornell AAP - MSAUD/ARCH 6131/Example Data/"
)

## Reading a Rhino file
Reading a Rhino file with `rhino3dm` is very easy! Much like reading a CSV or GeoJSON, you can use a helpful `Read()` method on a `File3dm` class.

Note that `rhino3dm`'s convention is to use [camel case or Pascal case](https://en.wikipedia.org/wiki/Camel_case) for naming. This means words have a first uppercase letter, like `NurbsCurveKnotList`. It's more common to see [snake case](https://en.wikipedia.org/wiki/Snake_case) in Python, which looks something like: `read_csv`. You'll need to be sure to capiitalize correctly when using `rhino3dm`.

In [9]:
# Let's read a 3DM file
# File3dm.Read() takes a string path, so you'll need to make sure your path is created correctly
rhino_file_path = EXAMPLE_DATA_DIRECTORY + "example.3dm"

print(f"Reading Rhino file from: '{EXAMPLE_DATA_DIRECTORY + 'example.3dm'}'")
example_3dm = rhino3dm.File3dm.Read(rhino_file_path)

Reading Rhino file from: '/content/drive/MyDrive/Cornell AAP - MSAUD/ARCH 6131/Example Data/example.3dm'


If you run into errors here, check your `rhino_file_path`. The `File3dm.Read()` method takes only a string as its argument for where to look for the file, rather than a smarter `Path` object. This means you have to make sure the full path is correctly written with a forward slash ("/") between directories.

There are some useful attributes on the `File3dm` class. You can read about them [here](https://mcneel.github.io/rhino3dm/python/api/File3dm.html). As you work with `rhino3dm` and other Python libraries, reading the docs will always be a good way to learn.

In [10]:
# There are some helpful attributes on the 3DM fiile
print(f"Unit systen: {example_3dm.Settings.ModelUnitSystem}")
print(f"Made by: {example_3dm.ApplicationName}")
print(f"File details: {example_3dm.ApplicationDetails}")

Unit systen: UnitSystem.Meters
Made by: Rhinoceros 7
File details: Evaluation, build 2023-08-09


## Accessing layers and geometry
So we've opened the file. What now? Usefully, we can take a look at more details about the data within the Rhino file. Let's try this with layers and geometry — the two essential building blocks for working with Rhino flies.

Layers are stored on a `File3dmLayerTable` object attached to each `File3dm`. You can access it with the `Layer` property.

In [11]:
# Let's look at the layers property on our Rhino flie.
example_3dm.Layers

`Layers` returns an iterable object. By itself, this isn't that useful to look at — it's a container for more data that we want.

Since the `File3dmLayerTable` is an iterable object, we can use a simple `for` loop to check out its contents. This will give us a single `Layer` object for each loop of the iteration. We can then access more meaningful attributes and properties on the layer.

I've printed out some below. Check out others in the [docs](https://mcneel.github.io/rhino3dm/python/api/Layer.html).

In [14]:
# Let's try iterating over the layers
print(f"There are {len(example_3dm.Layers)} layers in the Rhino file.\n")

# For each layer, you can get more information
for example_layer in example_3dm.Layers:
    print(f"Layer name: '{example_layer.Name}'")
    print(f"Full path: '{example_layer.FullPath}'")
    print(f"index: {example_layer.Index}")
    print(f"Visible: {example_layer.Visible}")
    print(f"Locked: {example_layer.Locked}")
    print(f"Color: {example_layer.Color}\n")

There are 5 layers in the Rhino file.

Layer name: 'Default'
Full path: 'Default'
index: 0
Visible: True
Locked: False
Color: (0, 0, 0, 255)

Layer name: '3D Geometry'
Full path: '3D Geometry'
index: 1
Visible: True
Locked: False
Color: (0, 0, 0, 255)

Layer name: 'Volumes'
Full path: '3D Geometry::Volumes'
index: 2
Visible: True
Locked: False
Color: (200, 0, 0, 255)

Layer name: 'Surfaces'
Full path: '3D Geometry::Surfaces'
index: 3
Visible: True
Locked: False
Color: (125, 38, 205, 255)

Layer name: 'Curves'
Full path: 'Curves'
index: 4
Visible: True
Locked: False
Color: (0, 0, 255, 255)



Try opening up the `example.3dm` file directly in Rhino and comparing it to the above. A few observations:

- Nested Rhino layers have a "::" separator in them between child and parent layers. So "Layer A::Layer B" means Layer B is the child layer of Layer A
- Each layer has a unique index
- Layers have `bool` values (short for boolean) for their visible and locked states: e.g. they can be either `True` or `False`.
- Layer colors are stored as tuples of RGBA type (look familiar?)

OK, now let's try to do the same with geometry. `File3dm.Objects` will give us `File3dmObjectTable`.

In [16]:
# Here's the object table
example_3dm.Objects

As, before we can iterate over this object as if it were a list. Note that each `File3dmObject` has two main properties: `Attributes` and `Geometry`.

Each stores related informatoin about the object.

In [18]:
# Let's try iterating over the geometry
print(f"There are {len(example_3dm.Objects)} objects in the Rhino file.\n")

for example_object in example_3dm.Objects:
    # For simpliciity, let's use variable to reference the attributes and geometry
    attributes = example_object.Attributes
    geometry = example_object.Geometry

    print(f"Geometry type: {geometry.ObjectType}")
    print(f"ID: '{attributes.Id}'")
    # We can get the name of the layer using the layer's index
    print(f"Layer: '{example_3dm.Layers.FindIndex(attributes.LayerIndex).Name}'\n")

There are 7 objects in the Rhino file.

Geometry type: ObjectType.Brep
ID: 'aa10934a-ad97-4af1-b0fa-6b40e38ac23b'
Layer: 'Surfaces'

Geometry type: ObjectType.Curve
ID: '9ede57be-3b5d-4ebd-ae6d-cbf61a330f9a'
Layer: 'Curves'

Geometry type: ObjectType.Curve
ID: '3b83f854-68be-4268-8cbf-e897fa04f9b2'
Layer: 'Curves'

Geometry type: ObjectType.Brep
ID: '5c417cf2-b825-45ae-ba4d-5aa8f7697b40'
Layer: 'Volumes'

Geometry type: ObjectType.Brep
ID: '1211cc07-8074-4a13-99e6-038c139d3198'
Layer: 'Surfaces'

Geometry type: ObjectType.Curve
ID: '6b3a5b45-af08-4cbe-9061-a01d2a48ff8e'
Layer: 'Curves'

Geometry type: ObjectType.Extrusion
ID: 'c0c3e292-2bbd-460e-9435-44528602fc4c'
Layer: 'Volumes'



Pretty cool, right? So we can see that each object has a type, an ID and we can find its layer through the layer index.

## Creating layers and geometry
Now let's try it in the reverse

In [176]:
# Make an empty filie
new_3dm = rhino3dm.File3dm()

In [177]:
# What's the unit system?
print(f"The file's units are in {new_3dm.Settings.ModelUnitSystem}")

# How many layers are in the empty file?
print(f"There are {len(new_3dm.Layers)} layers in the file")

The file's units are in UnitSystem.Millimeters
There are 0 layers in the file


In [178]:
# Let's set the units appropriately
new_3dm.Settings.ModelUnitSystem = rhino3dm.UnitSystem.Meters

In [179]:
# Let's set a "constant" variable for red
RED_RGBA = (255, 0, 0, 255)

# Add the layer with a name and color
new_3dm.Layers.AddLayer("My First Layer", RED_RGBA)

0

In [180]:
# Check the results
print(f"There are {len(new_3dm.Layers)} layers in the file")
print(f"Layers are: {[f'{l.FullPath} at index {l.Index}' for l in new_3dm.Layers]}")

There are 1 layers in the file
Layers are: ['My First Layer at index 0']


Layers are more straightforward than geometry.

Let's start with a simple `Point`.

In [181]:
# Point
# First, create a point geometry from a 3D location
test_pt = rhino3dm.Point(rhino3dm.Point3d(0, 0, 0))

In [182]:
# Let's define geometry attributes that we can re-use
test_attributes = rhino3dm.ObjectAttributes()

# This sets the attributes so that the objects is on the layer with index 0
test_attributes.LayerIndex = 0

In [183]:
# Add the geometry and object to the Rhino file
new_3dm.Objects.Add(test_pt, attributes)

UUID('5fef4c59-8954-b045-85bb-72b08628c622')

In [184]:
print(f"There are {len(new_3dm.Objects)} objects in the Rhino file.")

There are 1 objects in the Rhino file.


There's some nuance. You can only `Add()` geometry that is derived from `GeometryBase`:

In [185]:
rhino3dm.GeometryBase.__subclasses__()

[rhino3dm._rhino3dm.AnnotationBase,
 rhino3dm._rhino3dm.TextDot,
 rhino3dm._rhino3dm.InstanceReference,
 rhino3dm._rhino3dm.Hatch,
 rhino3dm._rhino3dm.PointCloud,
 rhino3dm._rhino3dm.Point,
 rhino3dm._rhino3dm.PointGrid,
 rhino3dm._rhino3dm.Curve,
 rhino3dm._rhino3dm.Mesh,
 rhino3dm._rhino3dm.Surface,
 rhino3dm._rhino3dm.SubD,
 rhino3dm._rhino3dm.Brep,
 rhino3dm._rhino3dm.Light]

In [186]:
# Line
# The line requires two 3D locatons for its start and end
test_line = rhino3dm.LineCurve(rhino3dm.Point3d(0, 0, 0), rhino3dm.Point3d(10, 10, 10))

# Polylne
# A polyline can be made from a list of points
test_polyline = rhino3dm.Polyline(
    [
        rhino3dm.Point3d(0, 0, 0),
        rhino3dm.Point3d(0, 10, 0),
        rhino3dm.Point3d(0, 10, 10),
        rhino3dm.Point3d(10, 10, 10),
    ]
).ToPolylineCurve()

# Planar polyline curve
# We create this by converting a polyline to a curve
test_polyline_curve = rhino3dm.Polyline(
    [
        rhino3dm.Point3d(0, 0, 0),
        rhino3dm.Point3d(-10, 0, 0),
        rhino3dm.Point3d(-10, -10, 0),
        rhino3dm.Point3d(0, -10, 0),
        rhino3dm.Point3d(0, 0, 0),
    ]
).ToPolylineCurve()

# Extrusion
# Finally, you can make a 3D extrusion from a polyline curve
test_extrusion = rhino3dm.Extrusion.Create(test_polyline_curve, height=20, cap=True)

In [187]:
test_polyline.IsClosed

False

In [188]:
ids_to_delete = [o.Attributes.Id for o in new_3dm.Objects]

print("Removing all other objects")
for id_to_delete in ids_to_delete:
    print(f"Removing object ID {id_to_delete} ...")
    new_3dm.Objects.Delete(id_to_delete)

Removing all other objects
Removing object ID 5fef4c59-8954-b045-85bb-72b08628c622 ...


In [190]:
objects_to_add = [
    test_pt,
    test_line,
    test_polyline,
    test_polyline_curve,
    test_extrusion,
]

for object_to_add in objects_to_add:
    print(f"Adding {object_to_add.ObjectType} to flie ...")
    new_3dm.Objects.Add(object_to_add, attributes)

Adding ObjectType.Point to flie ...
Adding ObjectType.Curve to flie ...
Adding ObjectType.Curve to flie ...
Adding ObjectType.Curve to flie ...
Adding ObjectType.Extrusion to flie ...


In [191]:
print(f"There are {len(new_3dm.Objects)} objects in the Rhino file.")

There are 5 objects in the Rhino file.


## Saving a Rhino flie
Saving is simple. Be sure to set your version appropriately.

In [192]:
new_3dm.Write("new_example.3dm", version=7)

True

Should look like:

![screenshot of Rhino]('Example Data/Screenshot 2023-09-03 at 6.11.31 PM.png')